In [39]:
import pandas as pd
import sys
sys.path.append('..')
from src.utils.db_engine import db_engine
engine = db_engine()

In [40]:
query = """
    SELECT 
        a.first_name,
        a.last_name,
        f.rating,
        COUNT(*) as movie_count
    FROM 
        film as f
    INNER JOIN 
        film_actor as fa
        ON 
            f.film_id = fa.film_id
    INNER JOIN 
        actor as a
        ON 
            fa.actor_id = a.actor_id
    GROUP BY 
        a.first_name, a.last_name, f.rating
    ORDER BY 
        movie_count DESC
"""

In [ ]:
df = pd.read_sql(query, engine)
df

In [41]:
#Figure out how much money each film made in rentals
query_film_money = """
    SELECT
        f.title,
        sum(p.amount)
    FROM
        film as f
    INNER JOIN 
        inventory as i
    ON
        f.film_id = i.film_id
    INNER JOIN 
        rental as r
    ON
        i.inventory_id = r.inventory_id
    INNER JOIN 
        payment as p
    ON
        r.rental_id = p.rental_id
    GROUP BY
        f.title
    ORDER BY 
        sum(p.amount) DESC
"""
pd.read_sql(query_film_money, engine)

,title,sum
0,Telegraph Voyage,215.75
1,Zorro Ark,199.72
2,Wife Turn,198.73
3,Innocent Usual,191.74
4,Hustler Party,190.78
...,...,...
953,Young Language,6.93
954,Freedom Cleopatra,5.95
955,Duffel Apocalypse,5.94
956,Oklahoma Jumanji,5.94


In [43]:
#Now find the actors in those films
best_selling_actors_query = """
    SELECT
        a.first_name,
        a.last_name,
        f.title,
        f.release_year,
        SUM(p.amount) as total_revenue
    FROM
        film as f
    INNER JOIN 
        film_actor as fa
    ON
        f.film_id = fa.film_id
    INNER JOIN 
        actor as a
    ON
        fa.actor_id = a.actor_id
    INNER JOIN 
        inventory as i
    ON
        f.film_id = i.film_id
    INNER JOIN 
        rental as r
    ON
        i.inventory_id = r.inventory_id
    INNER JOIN 
        payment as p
    ON
        r.rental_id = p.rental_id
    GROUP BY
        a.first_name, a.last_name, f.title, f.release_year
    ORDER BY 
        total_revenue DESC
"""
df_best_selling_actors = pd.read_sql(best_selling_actors_query, engine)
df_best_selling_actors

,first_name,last_name,title,release_year,total_revenue
0,Vivien,Basinger,Telegraph Voyage,2006,215.75
1,Lucille,Dee,Telegraph Voyage,2006,215.75
2,Thora,Temple,Telegraph Voyage,2006,215.75
3,Woody,Hoffman,Telegraph Voyage,2006,215.75
4,Gina,Degeneres,Telegraph Voyage,2006,215.75
...,...,...,...,...,...
5241,Rip,Crawford,Texas Watch,2006,5.94
5242,Groucho,Sinatra,Duffel Apocalypse,2006,5.94
5243,Cary,Mcconaughey,Oklahoma Jumanji,2006,5.94
5244,Rip,Crawford,Oklahoma Jumanji,2006,5.94


In [ ]:
df = None
with open("../src/sql/best_selling_actors_query.sql","r") as query:
    df = pd.read_sql_query(query.read(), engine)

df

In [ ]:
df_best_selling_actors[["first_name","last_name","total_revenue"]] \
    .groupby(["first_name","last_name"]) \
    .sum() \
    .reset_index() \
    .sort_values(by="total_revenue", ascending=False)

In [44]:
actor_year_revenue = df_best_selling_actors \
    .groupby([
        "first_name",
        "last_name",
        "release_year"])["total_revenue"] \
    .sum() \
    .reset_index()
best_actor_per_year = actor_year_revenue \
    .loc[actor_year_revenue.groupby(["release_year"])["total_revenue"].idxmax()]

best_actor_per_year

,first_name,last_name,release_year,total_revenue
180,Susan,Davis,2006,3193.49
